In [146]:
import time
start_time = time.time()

In [147]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd
from bs4 import BeautifulSoup as bs4
import re

In [148]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [C:\Users\saidu\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [149]:
# Saving Query Data from csv to list
def savingQueryData(csvFile):
    queryDataCsv= pd.read_csv(csvFile)
    queryList = [i for i in queryDataCsv["query"]]
    locationList = [i for i in queryDataCsv["location"]]
    return queryList, locationList
    
def savingLoginData(csvFile):
    loginDataCsv = pd.read_csv(csvFile)
    emailList = [i for i in loginDataCsv["email"]]
    passwordList = [i for i in loginDataCsv["password"]]
    return emailList, passwordList

# Return searchable facebook url
def facebook_search_url(queryList, locationList):
    search_url_list = []
    for j in range(len(queryList)):
        splitQuery = queryList[j].split(" ")
        splitLocation = locationList[j].split(" ")

        searchUrl = "https://www.facebook.com/search/pages/?q="
        for i in splitQuery:
            searchUrl = searchUrl + i + "%20"
        for i in splitLocation:
            searchUrl = searchUrl + i + "%20"
        search_url_list.append(searchUrl)
    return search_url_list

queryList, locationList = savingQueryData("queries.csv")
search_url_list = facebook_search_url(queryList, locationList)

search_url_list

['https://www.facebook.com/search/pages/?q=car%20repair%20new%20york%20',
 'https://www.facebook.com/search/pages/?q=car%20repair%20canada%20',
 'https://www.facebook.com/search/pages/?q=house%20canada%20',
 'https://www.facebook.com/search/pages/?q=house%20new%20york%20',
 'https://www.facebook.com/search/pages/?q=hotel%20%20boston%20']

In [228]:
def generalInfo_aboutPage():
    soup = bs4(driver.page_source, "html.parser")
    list_info = soup.find_all("div", class_="je60u5p8")

    info = ""
    for idx, j in enumerate(list_info):
        for i in list_info[idx].findAll("span"):
            if not i.text in info:
                info = info + i.text + " "
        info = info.replace("General", "")
    return info

def collectURL(string):
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)      
    return [x[0] for x in url]

def collectEMAIL(string):
    match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', string)
    return match
def collectPHONE(string):
    match = re.findall(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', string)
    return match

def mainScraper():
    #Html of about page
    html = driver.page_source
    soup = bs4(html, "html.parser")
    target_listing = soup.find_all("div", class_="sjgh65i0")

    master_list = []
    for idx,i in enumerate(target_listing):
    # Single Listing
        data_dict={}
        try:
            page_name = i.a["aria-label"]
            facebook_link = i.a["href"]
            data_dict["Page Name"] = page_name
            data_dict["Facebook Link"] = facebook_link
            facebook_link_aboutPage = facebook_link.replace("?__tn__=%3C", "about")
            if not "/about" in facebook_link_aboutPage:
                facebook_link_aboutPage = facebook_link_aboutPage.replace("about", "/about")
            print(facebook_link_aboutPage, "\n")
            if "facebook.com" in facebook_link_aboutPage:
                driver.get(facebook_link_aboutPage)
            time.sleep(5)
            #Accessing each lead's about page
            data_dict["Url"] = "".join(collectURL(generalInfo_aboutPage()))
            data_dict["Email"] = "".join(collectEMAIL(generalInfo_aboutPage()))
            data_dict["Phone Numer"] = "".join(collectPHONE(generalInfo_aboutPage()))
            data_dict["General Information"] = generalInfo_aboutPage()
        except:
            print("Error")
        master_list.append(data_dict)
        time.sleep(5)
        print(f"Percentage done: {idx/len(target_listing)*100}%")
    df = pd.DataFrame(master_list)
    return df
    

In [194]:
emailList, passwordList = savingLoginData("login.csv")

driver.get("https://www.facebook.com")
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

username.clear()
username.send_keys(emailList[0])
password.clear()
password.send_keys(passwordList[0])

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

TimeoutException: Message: 


In [ ]:
time.sleep(5)
for idx,url in enumerate(search_url_list):
    driver.get(url)
    time.sleep(3)
    for i in range(10):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") 
#     TODO: Solution for House Canda Sraping - Posts/Pages/Group ?
    time.sleep(2)
    df = mainScraper()
    queryList, locationList = savingQueryData("queries.csv")
    df.to_csv(f"{queryList[idx]} {locationList[idx]}.csv")
    break

https://web.facebook.com/Car-Repair-NYC-New-York-City-Auto-Repair-Manhattan-NY-248902591857537/about 

Percentage done: 0.0%
https://web.facebook.com/nycarassociation/about 

Percentage done: 0.9803921568627451%
https://web.facebook.com/Windshield-Replacement-Auto-Glass-Repair-New-York-NY-167774173367610/about 

Percentage done: 1.9607843137254901%
https://web.facebook.com/Certified-Auto-Repair-of-New-York-Mills-Inc-282732623054/about 

Percentage done: 2.941176470588235%
https://web.facebook.com/nydaily/about 

Percentage done: 3.9215686274509802%
Error
Percentage done: 4.901960784313726%
https://web.facebook.com/Best-Auto-Repair-NYC-Towing-Service-Uptown-Auto-Repair-New-York-112361068825211/about 

Percentage done: 5.88235294117647%
https://web.facebook.com/International-2000-Auto-Repair-Inc-179616865394006/about 

Percentage done: 6.862745098039216%
https://web.facebook.com/AutoClinicofMahopac/about 

Percentage done: 7.8431372549019605%
https://web.facebook.com/BarnettAutoRepair/ab

In [140]:
driver.quit()

In [141]:
print("Time Taken",  (time.time() - start_time)/60, " min")

Time Taken 28.87259211540222  min


In [230]:
df

,Page Name,Facebook Link,Url,Email,Phone Numer,General Information
0,Car Repair NYC - New York City Auto Repair - M...,https://web.facebook.com/Car-Repair-NYC-New-Yo...,http://www.uptownautorepair.net/nychttp://www....,,212-876-1040876-1047286-8596876-1047286-8596,54 people like this 58 people follow this 5 p...
1,New York City Auto Repair Association,https://web.facebook.com/nycarassociation/?__t...,https://www.nycara.org/,info@nycara.org,917-522-5912,66 people like this 68 people follow this Cha...
2,Windshield Replacement Auto Glass Repair New Y...,https://web.facebook.com/Windshield-Replacemen...,http://www.windshieldreplacementautoglassrepai...,,212-203-4819,2 people like this 2 people follow this 1 per...
3,"Certified Auto Repair of New York Mills, Inc.",https://web.facebook.com/Certified-Auto-Repair...,http://locations.autovalue.com/mn/new%20york%2...,certautorepair@lakesplus.com,218-385-4488,577 people like this 591 people follow this 3...
4,New York City Photos,https://web.facebook.com/nydaily?__tn__=%3C,,,,
